In [1]:
import pandas as pd
import numpy as numpy
from importlib import reload
from tqdm import tqdm_notebook as tqdm
import time
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import norm

import pdaactconn as pc
from trialexplorer import AACTStudySet

In [2]:
conn = pc.AACTConnection(source=pc.AACTConnection.REMOTE)
ss = AACTStudySet.AACTStudySet(conn=conn, 
                               tqdm_handler=tqdm)
ss.add_constraint("study_type = 'Interventional'")
ss.add_constraint("results_first_submitted_date is not null")
ss.add_constraint("enrollment_type = 'Actual'")
ss.add_constraint("enrollment >= 10")
ss.add_constraint("enrollment <= 500")
ss.add_constraint("phase = 'Phase 2'")
ss.load_studies()

8780 studies loaded!


In [3]:
ss.add_dimensions(['baseline_measurements', 'result_groups', 'outcomes', 'outcome_measurements',
                   'designs', 'eligibilities'])
ss.refresh_dim_data()
rg = ss.dimensions['result_groups']
bm = ss.dimensions['baseline_measurements']
om = ss.dimensions['outcome_measurements']
out = ss.dimensions['outcomes']
des = ss.dimensions['designs']

Successfuly added these 6 dimensions: ['baseline_measurements', 'result_groups', 'outcomes', 'outcome_measurements', 'designs', 'eligibilities']
Failed to add these 0 dimensions: []


Syncing the temp table temp_cur_studies in 18 chunks x 500 records each

Creating index on the temp table
 - Loading dimension baseline_measurements
 -- Loading raw data
 -- Sorting index
 - Loading dimension result_groups
 -- Loading raw data
 -- Sorting index
 - Loading dimension outcomes
 -- Loading raw data
 -- Sorting index
 - Loading dimension outcome_measurements
 -- Loading raw data
 -- Sorting index
 - Loading dimension designs
 -- Loading raw data
 -- Sorting index
 - Loading dimension eligibilities
 -- Loading raw data
 -- Sorting index


In [4]:
des.data.allocation.value_counts(dropna=False)

Randomized        4921
NaN               2921
Non-Randomized     938
Name: allocation, dtype: int64

In [5]:
des.data.allocation.fillna('Missing', inplace=True)

In [6]:
des.data.allocation.value_counts()

Randomized        4921
Missing           2921
Non-Randomized     938
Name: allocation, dtype: int64

In [7]:
#Get studies with exactly two treatment groups besides "Total" and grab necessary columns
combined_measures = pd.merge(rg.data, bm.data, left_on = ['nct_id', 'id'], right_on = ['nct_id', 'result_group_id'])
combined_measures = combined_measures[combined_measures.title_x != 'Total']
num_groups = combined_measures.groupby('nct_id').ctgov_group_code_x.nunique()
num_groups = pd.DataFrame(num_groups)
num_groups.columns.values[0] = 'n_groups'
combined_measures = combined_measures.merge(num_groups, on=['nct_id'])
combined_measures = combined_measures[combined_measures.n_groups==2]
print(combined_measures.shape)
#Limit to randomized studies
combined_measures = combined_measures.merge(des.data, on = 'nct_id')
combined_measures = combined_measures[combined_measures.allocation == 'Randomized']
print(combined_measures.shape)
study_balance_dat = combined_measures[['ctgov_group_code_x', 'classification', 'category', 'title_y',
                                       'param_type', 'param_value_num',
                                       'dispersion_type', 'dispersion_value_num']]

(61684, 22)
(51574, 35)


In [8]:
study_balance_dat.title_y.value_counts().head(10)
#We'll go with age and sex for now

Age                                                             10044
Sex: Female, Male                                                9920
Race (NIH/OMB)                                                   9198
Region of Enrollment                                             4320
Race/Ethnicity, Customized                                       3052
Ethnicity (NIH/OMB)                                              2844
Age, Customized                                                  1230
Gender                                                            348
Eastern Cooperative Oncology Group (ECOG) Performance Status      178
Weight                                                            128
Name: title_y, dtype: int64

In [9]:
sb_age = study_balance_dat[study_balance_dat.title_y == 'Age']
sb_age_cat = sb_age[sb_age.param_type=='Count of Participants']
sb_age_con = sb_age[sb_age.param_type=='Mean']
print(sb_age_cat.index.nunique())
print(sb_age_con.index.nunique())
#We'll choose the continuous version for now, since it has more

914
1935


In [10]:
#Sex is always a count variable; only variability is capitalization of "category" field
#Get rid of rows that aren't sex or age
age_condition1 = study_balance_dat.title_y == 'Age'
age_condition2 = study_balance_dat.param_type == 'Mean'
sex_condition = study_balance_dat.title_y == 'Sex: Female, Male'
full_condition = (age_condition1 & age_condition2) | sex_condition
small_balance_dat = study_balance_dat[full_condition]
small_balance_dat.head(5)

,ctgov_group_code_x,classification,category,title_y,param_type,param_value_num,dispersion_type,dispersion_value_num
nct_id,,,,,,,,
NCT00000392,B2,,Male,"Sex: Female, Male",Count of Participants,104.0,,NaN
NCT00000392,B2,,Female,"Sex: Female, Male",Count of Participants,5.0,,NaN
NCT00000392,B1,,Male,"Sex: Female, Male",Count of Participants,101.0,,NaN
NCT00000392,B1,,Female,"Sex: Female, Male",Count of Participants,5.0,,NaN
NCT00001304,B2,,Male,"Sex: Female, Male",Count of Participants,4.0,,NaN


In [11]:
#get studies that have both measures we're using
num_measures = small_balance_dat.groupby('nct_id').title_y.nunique()
num_measures = pd.DataFrame(num_measures)
num_measures.columns.values[0] = 'n_measures'
small_balance_dat = small_balance_dat.merge(num_measures, on=['nct_id'])
small_balance_dat = small_balance_dat[small_balance_dat.n_measures==2]

#assert that each study has 6 rows now--2 sex and 1 age for each of the 2 arms
num_rows = small_balance_dat.groupby('nct_id').title_y.count()
num_rows = pd.DataFrame(num_rows)
num_rows.columns.values[0] = 'n_rows'
small_balance_dat = small_balance_dat.merge(num_rows, on=['nct_id'])
small_balance_dat = small_balance_dat[small_balance_dat.n_rows == 6]
print(small_balance_dat.index.nunique())

1841


In [12]:
#handle aforementioned capitalization issue
small_balance_dat.category = small_balance_dat.category.str.lower()
small_balance_dat.head()

,ctgov_group_code_x,classification,category,title_y,param_type,param_value_num,dispersion_type,dispersion_value_num,n_measures,n_rows
nct_id,,,,,,,,,,
NCT00001586,B2,,male,"Sex: Female, Male",Count of Participants,35.00,,NaN,2,6
NCT00001586,B2,,female,"Sex: Female, Male",Count of Participants,21.00,,NaN,2,6
NCT00001586,B2,,,Age,Mean,58.42,Standard Deviation,11.32,2,6
NCT00001586,B1,,male,"Sex: Female, Male",Count of Participants,30.00,,NaN,2,6
NCT00001586,B1,,female,"Sex: Female, Male",Count of Participants,19.00,,NaN,2,6


In [13]:
def calculate_imbalance(study_frame):
    nctid = study_frame.index[0]
    group_codes = study_frame.ctgov_group_code_x.unique()
    group1 = study_frame[study_frame.ctgov_group_code_x == group_codes[0]]
    group2 = study_frame[study_frame.ctgov_group_code_x == group_codes[1]]
    
    sex_imbalance, group_size_imbalance = calculate_sex_imbalance(group1, group2)
    age_imbalance = calculate_age_imbalance(group1, group2)
    
    return([nctid, sex_imbalance, age_imbalance, group_size_imbalance])

def calculate_sex_imbalance(arm1, arm2):
    arm1_sex = arm1[arm1.title_y == 'Sex: Female, Male']
    arm2_sex = arm2[arm2.title_y == 'Sex: Female, Male']
    
    arm1_size = arm1_sex.param_value_num.sum()
    arm2_size = arm2_sex.param_value_num.sum()
    
    arm1_men = arm1_sex[arm1_sex.category == 'male'].param_value_num[0]
    arm2_men = arm2_sex[arm2_sex.category == 'male'].param_value_num[0]
    
    arm1_pct_men = arm1_men / arm1_size
    arm2_pct_men = arm2_men / arm2_size
    
    sex_imbalance = abs(arm1_pct_men - arm2_pct_men)
    group_size_imbalance = abs(0.5 - (arm1_size) / (arm1_size + arm2_size))
    return((sex_imbalance, group_size_imbalance))

def calculate_age_imbalance(arm1, arm2):
    arm1_mean_age = arm1[arm1.title_y == 'Age'].param_value_num[0]
    arm2_mean_age = arm2[arm2.title_y == 'Age'].param_value_num[0]
    
    age_imbalance = 12 * abs(arm1_mean_age - arm2_mean_age) 
    return(age_imbalance)

In [14]:
#Time to calculate imbalances
imbalance_dat = []
study_ids = small_balance_dat.index.unique()
n_studies = study_ids.shape[0]
print('Calculating imbalance for ' + str(n_studies) + ' studies')
for i in range(n_studies):
    current_id = study_ids[i]
    current_study = small_balance_dat[small_balance_dat.index == current_id]
    current_imbalances = calculate_imbalance(current_study)
    imbalance_dat.append(current_imbalances)
    if (i + 1) % 1000 == 0:
        print('Finished with ' + str(i + 1) + ' studies.')
imbalance_frame = pd.DataFrame(imbalance_dat, columns = ['nct_id', 'sex_imbalance', 'age_imbalance', 'group_size_imbalance'])
imbalance_frame.head()

Calculating imbalance for 1841 studies
Finished with 1000 studies.


,nct_id,sex_imbalance,age_imbalance,group_size_imbalance
0,NCT00001586,0.012755,26.64,0.033333
1,NCT00001596,0.152174,53.28,0.157143
2,NCT00001723,0.010000,1.56,0.000000
3,NCT00003222,0.217033,24.00,0.182927
4,NCT00004980,0.014493,4.80,0.040000


In [15]:
imbalance_frame.sex_imbalance.describe()

count    1841.000000
mean        0.080448
std         0.095739
min         0.000000
25%         0.010638
50%         0.052288
75%         0.111111
max         0.750000
Name: sex_imbalance, dtype: float64

In [16]:
imbalance_frame.age_imbalance.describe()

count    1841.000000
mean       25.597808
std        30.253860
min         0.000000
25%         6.840000
50%        16.800000
75%        34.800000
max       463.200000
Name: age_imbalance, dtype: float64

In [17]:
om.data.reset_index(inplace=True)
out.data.reset_index(inplace=True)
rg.data.reset_index(inplace=True)
out.data = out.data.add_prefix('out_')
om.data = om.data.add_prefix('om_')
rg.data = rg.data.add_prefix('rg_')

In [18]:
out.data.head()

,out_nct_id,out_id,out_outcome_type,out_title,out_description,out_time_frame,out_population,out_anticipated_posting_date,out_anticipated_posting_month_year,out_units,out_units_analyzed,out_dispersion_type,out_param_type
0,NCT00000392,306115,Secondary,Change in Neurocognitive Performance Domain z ...,Higher values for change in z-score represent ...,Baseline and 6 months,,None,,z score,,Standard Error,Mean
1,NCT00000392,306116,Primary,Change in Global Neurocognitive Performance z ...,Higher values for change in z-score represent ...,Baseline and 6 months,,None,,z score,,Standard Error,Mean
2,NCT00001213,306175,Primary,Number of Eyes With a Corneal Cystine Crystal ...,Response is defined as a decrease from baselin...,Any Time Point Up to 19 Years,One hundred sixty-one (161) participants were ...,None,,eyes,Participants,,Number
3,NCT00001213,306176,Primary,Number of Participants With Serious and Non-Se...,Since efficacy of ophthalmic cysteamine was es...,Any Time Point up to 27 Years,,None,,participants,,,Number
4,NCT00001304,306161,Primary,Urine Calcium Excretion Level,Measurements were taken1 hour before the morni...,3 years,All patients on the study,None,,mmol/24 h,,Standard Deviation,Mean


In [19]:
combined_outcomes = om.data.merge(out.data, left_on = ['om_nct_id', 'om_outcome_id'], 
                                  right_on = ['out_nct_id', 'out_id'])

combined_outcomes = combined_outcomes.merge(rg.data, left_on = ['om_nct_id','om_result_group_id'], 
                                            right_on = ['rg_nct_id','rg_id'])
primary_outcomes = combined_outcomes[combined_outcomes.out_outcome_type == 'Primary']

In [20]:
primary_outcomes.om_param_type.value_counts()

Number                          52951
Count of Participants           48707
Mean                            38140
Median                           7244
Geometric Mean                   5319
Least Squares Mean               3663
Geometric Least Squares Mean      299
Log Mean                           42
Count of Units                     21
                                    3
Name: om_param_type, dtype: int64

In [21]:
primary_outcomes = primary_outcomes[primary_outcomes.om_param_type == 'Mean']
primary_outcomes.shape

(38140, 38)

In [22]:
primary_outcomes = primary_outcomes[primary_outcomes.om_param_value != 'NA']
primary_outcomes.shape

(37853, 38)

In [23]:
num_groups = primary_outcomes.groupby(['om_nct_id', 'om_outcome_id']).om_result_group_id.nunique()
num_groups = pd.DataFrame(num_groups)
num_groups.columns.values[0] = 'n_groups'
primary_outcomes = primary_outcomes.merge(num_groups, on=['om_nct_id', 'om_outcome_id'])
primary_outcomes = primary_outcomes[primary_outcomes.n_groups==2]
primary_outcomes.shape

(10315, 39)

In [24]:
study_means = []
primary_ids = primary_outcomes.om_nct_id.unique()
for pid in primary_ids:
    temp_frame = primary_outcomes[primary_outcomes.om_nct_id == pid]
    temp_outcome_ids = temp_frame.om_outcome_id.unique()
    sampled_index = numpy.random.choice(temp_outcome_ids.shape[0], 1, replace=False)
    chosen_outcome = temp_frame[temp_frame.om_outcome_id == temp_outcome_ids[sampled_index][0]].reset_index()
    chosen_means = chosen_outcome.om_param_value_num
    study_means.append([pid, chosen_means[0], chosen_means[1]])
study_means = pd.DataFrame(study_means, columns = ['nct_id', 'mean1', 'mean2'])

In [25]:
study_means['abs_mean_dif'] = abs(study_means.mean1 - study_means.mean2)
study_means['abs_mean_sum'] = abs(study_means.mean1) + abs(study_means.mean2)
study_means['alt_test_statistic'] = study_means.abs_mean_dif / study_means.abs_mean_sum
study_means.shape

(1164, 6)

In [26]:
study_means.alt_test_statistic.describe([.05, .1, .25, .50, .75, .90, .95])

count    1164.000000
mean        0.317096
std         0.340560
min         0.000000
5%          0.006781
10%         0.015228
25%         0.045041
50%         0.164702
75%         0.499264
90%         1.000000
95%         1.000000
max         1.000000
Name: alt_test_statistic, dtype: float64

In [27]:
study_means.head()

,nct_id,mean1,mean2,abs_mean_dif,abs_mean_sum,alt_test_statistic
0,NCT00000392,0.16,0.24,0.08,0.40,0.200000
1,NCT00001304,1.92,2.00,0.08,3.92,0.020408
2,NCT00001596,-20.93,-23.52,2.59,44.45,0.058268
3,NCT00001723,-0.06,-0.12,0.06,0.18,0.333333
4,NCT00003895,1.34,0.10,1.24,1.44,0.861111


In [28]:
condition1 = study_means.alt_test_statistic > 0
condition2 = study_means.alt_test_statistic < 3
full_condition = condition1 & condition2
study_means = study_means[full_condition]
study_means.shape

(1141, 6)

In [29]:
regression_data = study_means.merge(imbalance_frame, on = ['nct_id'])
regression_data.shape
#This is about 2x as many studies as the old way

(728, 9)

In [30]:
regression_data.head()

,nct_id,mean1,mean2,abs_mean_dif,abs_mean_sum,alt_test_statistic,sex_imbalance,age_imbalance,group_size_imbalance
0,NCT00001596,-20.93,-23.52,2.59,44.45,0.058268,0.152174,53.28,0.157143
1,NCT00001723,-0.06,-0.12,0.06,0.18,0.333333,0.010000,1.56,0.000000
2,NCT00004980,8.61,5.85,2.76,14.46,0.190871,0.014493,4.80,0.040000
3,NCT00005669,-0.07,-0.11,0.04,0.18,0.222222,0.072260,3.60,0.030000
4,NCT00005879,-0.80,-1.10,0.30,1.90,0.157895,0.000000,2.40,0.007538


In [31]:
linmod = smf.ols('alt_test_statistic ~ age_imbalance + sex_imbalance', data=regression_data).fit()
print(linmod.summary())

                            OLS Regression Results                            
Dep. Variable:     alt_test_statistic   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.5170
Date:                Mon, 02 Dec 2019   Prob (F-statistic):              0.597
Time:                        21:35:14   Log-Likelihood:                -239.57
No. Observations:                 728   AIC:                             485.1
Df Residuals:                     725   BIC:                             498.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         0.3049      0.019     15.791